In [1]:
# Importa librerías
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Verifica la GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
print(f"Versión de PyTorch: {torch.__version__}")
print(f"Versión de CUDA: {torch.version.cuda}")

# Define el modelo LSTM (mismo que en Fase 3)
class LSTMModel(nn.Module):
    def __init__(self, input_size=30, hidden_size=64, num_layers=2, num_classes=6):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Carga el modelo entrenado
model = LSTMModel(input_size=30, hidden_size=64, num_layers=2, num_classes=6).to(device)
model.load_state_dict(torch.load('../models/lstm_model.pth'))
model.eval()
print("Modelo cargado exitosamente.")

# Carga los datos enriquecidos
data = pd.read_csv('../data/processed/physical_exercise_enriched.csv')
print("Primeras 5 filas del dataset:")
print(data.head())

Usando dispositivo: cuda
Versión de PyTorch: 2.8.0+cu126
Versión de CUDA: 12.6
Modelo cargado exitosamente.
Primeras 5 filas del dataset:
   pose_id    x_nose  x_left_eye_inner  x_left_eye  x_left_eye_outer  \
0      659  0.005001          0.011026    0.010531          0.017030   
1      660  0.005643          0.009842    0.009092          0.018570   
2      661 -0.080741         -0.111531   -0.083575         -0.111369   
3      662  0.011821          0.024027    0.020280          0.030158   
4      663 -0.004911         -0.006103    0.000368          0.007008   

   x_right_eye_inner  x_right_eye  x_right_eye_outer  x_left_ear  x_right_ear  \
0           0.000794    -0.002900          -0.005019    0.022900    -0.011029   
1           0.001104    -0.001759          -0.003827    0.018725    -0.009088   
2          -0.083769    -0.111719          -0.083855   -0.107371    -0.081137   
3           0.011744     0.013145           0.008056    0.035715     0.004197   
4          -0.019879    

Pérdida y precisión de entrenamiento/validación

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Preprocesamiento de datos
def create_sequences(data, seq_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i + seq_length][['x_nose', 'y_nose', 'x_left_shoulder', 'y_left_shoulder', 
                                          'x_right_shoulder', 'y_right_shoulder', 'x_left_elbow', 'y_left_elbow',
                                          'x_right_elbow', 'y_right_elbow', 'x_left_wrist', 'y_left_wrist',
                                          'x_right_wrist', 'y_right_wrist', 'x_left_hip', 'y_left_hip',
                                          'x_right_hip', 'y_right_hip', 'x_left_knee', 'y_left_knee',
                                          'x_right_knee', 'y_right_knee', 'x_left_ankle', 'y_left_ankle',
                                          'x_right_ankle', 'y_right_ankle', 'right_elbow_angle',
                                          'left_elbow_angle', 'right_knee_angle', 'left_knee_angle']].values
        target = data.iloc[i + seq_length]['pose']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Crea secuencias
seq_length = 10
sequences, targets = create_sequences(data, seq_length)

# Codifica las etiquetas
le = LabelEncoder()
targets_encoded = le.fit_transform(targets.ravel())

# Divide en entrenamiento y validación (80/20)
X_train, X_val, y_train, y_val = train_test_split(sequences, targets_encoded, test_size=0.2, random_state=42, stratify=targets_encoded)

# Crea Datasets
class ExerciseDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.FloatTensor(sequences)
        self.targets = torch.LongTensor(targets)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

train_dataset = ExerciseDataset(X_train, y_train)
val_dataset = ExerciseDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Evalúa el modelo
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Calcula métricas
train_loss, train_accuracy = evaluate_model(model, train_loader)
val_loss, val_accuracy = evaluate_model(model, val_loader)
print(f"Pérdida entrenamiento: {train_loss:.4f}, Precisión entrenamiento: {train_accuracy:.2f}%")
print(f"Pérdida validación: {val_loss:.4f}, Precisión validación: {val_accuracy:.2f}%")

Pérdida entrenamiento: 0.0012, Precisión entrenamiento: 100.00%
Pérdida validación: 0.0013, Precisión validación: 100.00%


Predicción con nuevos datos (despliegue básico)

In [3]:
# Predicción con una secuencia nueva (ejemplo)
def predict_sequence(model, sequence, le):
    model.eval()
    sequence = torch.FloatTensor(sequence).unsqueeze(0).to(device)  # Añade dimensión de batch
    with torch.no_grad():
        output = model(sequence)
        _, predicted = torch.max(output, 1)
        predicted_label = le.inverse_transform([predicted.cpu().numpy()])[0]
    return predicted_label

# Ejemplo de secuencia (usa los primeros 10 frames del dataset como prueba)
example_sequence = data.iloc[:10][['x_nose', 'y_nose', 'x_left_shoulder', 'y_left_shoulder', 
                                  'x_right_shoulder', 'y_right_shoulder', 'x_left_elbow', 'y_left_elbow',
                                  'x_right_elbow', 'y_right_elbow', 'x_left_wrist', 'y_left_wrist',
                                  'x_right_wrist', 'y_right_wrist', 'x_left_hip', 'y_left_hip',
                                  'x_right_hip', 'y_right_hip', 'x_left_knee', 'y_left_knee',
                                  'x_right_knee', 'y_right_knee', 'x_left_ankle', 'y_left_ankle',
                                  'x_right_ankle', 'y_right_ankle', 'right_elbow_angle',
                                  'left_elbow_angle', 'right_knee_angle', 'left_knee_angle']].values
real_label = data.iloc[10]['pose']
predicted_label = predict_sequence(model, example_sequence, le)
print(f"Etiqueta real: {real_label}")
print(f"Etiqueta predicha: {predicted_label}")

Etiqueta real: pushups_down
Etiqueta predicha: pushups_down


c:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\.venv\lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
